In [13]:
from Cadenas import *
from gtts import gTTS

In [43]:
def procesa_cadena(ltx=None,symbs=['x']):
    if not ltx or not symbs:
        return
    d = valida_ltx(ltx, symbs, funciones)
    cad = d['cad']
    a_cad = ajusta_cadena(cad,symbs) #procesa la cadena
    t = cad2tree(a_cad)
    pos = 0
    e = tree2expr(t,pos,symbs)
    return t,e

In [3]:
# Seccion obtenida de: https://www.lawebdelprogramador.com/foros/Python/297286-Transformar-numeros-a-literales.html
def numero_to_letras(numero):
	indicador = [("",""),("MIL","MIL"),("MILLON","MILLONES"),("MIL","MIL"),("BILLON","BILLONES")]
	entero = int(numero)
	decimal = int(round((numero - entero)*100))
	#print 'decimal : ',decimal 
	contador = 0
	numero_letras = ""
	while entero >0:
		a = entero % 1000
		if contador == 0:
			en_letras = convierte_cifra(a,1).strip()
		else :
			en_letras = convierte_cifra(a,0).strip()
		if a==0:
			numero_letras = en_letras+" "+numero_letras
		elif a==1:
			if contador in (1,3):
				numero_letras = indicador[contador][0]+" "+numero_letras
			else:
				numero_letras = en_letras+" "+indicador[contador][0]+" "+numero_letras
		else:
			numero_letras = en_letras+" "+indicador[contador][1]+" "+numero_letras
		numero_letras = numero_letras.strip()
		contador = contador + 1
		entero = int(entero / 1000)
	numero_letras = numero_letras
	return numero_letras

#Seccion obtenida de: https://www.lawebdelprogramador.com/foros/Python/297286-Transformar-numeros-a-literales.html
def convierte_cifra(numero,sw):
    lista_centana = ["",("CIEN","CIENTO"),"DOSCIENTOS","TRESCIENTOS","CUATROCIENTOS","QUINIENTOS","SEISCIENTOS","SETECIENTOS","OCHOCIENTOS","NOVECIENTOS"]
    lista_decena = ["",("DIEZ","ONCE","DOCE","TRECE","CATORCE","QUINCE","DIECISEIS","DIECISIETE","DIECIOCHO","DIECINUEVE"),
                    ("VEINTE","VEINTI"),("TREINTA","TREINTA Y "),("CUARENTA" , "CUARENTA Y "),
                    ("CINCUENTA" , "CINCUENTA Y "),("SESENTA" , "SESENTA Y "),
                    ("SETENTA" , "SETENTA Y "),("OCHENTA" , "OCHENTA Y "),
                    ("NOVENTA" , "NOVENTA Y ")
                ]
    lista_unidad = ["",("UN" , "UNO"),"DOS","TRES","CUATRO","CINCO","SEIS","SIETE","OCHO","NUEVE"]
    centena = int (numero / 100)
    decena = int((numero -(centena * 100))/10)
    unidad = int(numero - (centena * 100 + decena * 10))
    #print "centena: ",centena, "decena: ",decena,'unidad: ',unidad

    texto_centena = ""
    texto_decena = ""
    texto_unidad = ""

    #Validad las centenas
    texto_centena = lista_centana[centena]
    if centena == 1:
        if (decena + unidad)!=0:
            texto_centena = texto_centena[1]
        else :
            texto_centena = texto_centena[0]

    #Valida las decenas
    texto_decena = lista_decena[decena]
    if decena == 1 :
        texto_decena = texto_decena[unidad]
    elif decena > 1 :
        if unidad != 0 :
            texto_decena = texto_decena[1]
        else:
            texto_decena = texto_decena[0]
    #Validar las unidades
    #print "texto_unidad: ",texto_unidad
    if decena != 1:
        texto_unidad = lista_unidad[unidad]
        if unidad == 1:
            texto_unidad = texto_unidad[sw]

    return "%s %s %s" %(texto_centena,texto_decena,texto_unidad)


In [19]:
def numero_to_exponente(numero):
    dic = {
        2: 'AL CUADRADO',
        3: 'AL CUBO',
        4: 'A LA CUARTA POTENCIA',
        5: 'A LA QUINTA POTENCIA',
        6: 'A LA SEXTA POTENCIA',
        7: 'A LA SEPTIMA POTENCIA',
        8: 'A LA OCTAVA POTENCIA',
        9: 'A LA NOVENA POTENCIA',
    }
    return dic[numero]

In [74]:
'''
Funcion simbolo_a_texto
Argumentos: 
    *valor: Valor requerido para convertir en texto.

Funcionamiento:
    Esta funcion se encarga de convertir los simbolos en texto para:
        Cuando la cadena incluye tangente,seno, coseno, etc: Nodo actual, Cadena del hijo izquierdo, Cadena del hijo derecho
        En los demas casos: Cadena del hijo izquierdo, Nodo actual, Cadena del hijo derecho
'''

def simbolo_a_texto(valor):
    if not valor or valor == '0':
        return  ''
    simbolo = {
        '*':'por',
        '+':'mas',
        '/':'entre',
        '-':'menos',
        'tan': 'tangente de',
        'cos': 'coseno de',
        'sin': 'seno de',
        'atan': 'arcotangente de',
        'asin': 'arcoseno de',
        'acos': 'arcocoseno de',
        'sqrt': 'raiz cuadrada de',
        'cot': 'cotangente de',
        'csc': 'cosecante de',
        'sec': 'secante de',
        'exp': 'exponencial de',
        'ln': 'logaritmo natural de',
        'log': 'logaritmo de'
    }
    if valor in simbolo.keys():
        return simbolo[valor]
    else:
        if '^' in valor:
            valor_n = valor.replace('^(','')
            valor_n = valor_n.replace(')','')
            return f'{numero_to_exponente(int(valor_n))}'
        else:
            return valor
            

In [75]:
'''
Funcion obten_cadena
Argumentos: 
    *index: Indice donde se comienza el recorrido del arbol.
    *tree: El arbol a recorrer
    *cadena: Cadena que se va armando

Funcionamiento:
    Esta funcion se encarga de recorrer el arbol en cuestion, se obtienen las cadenas de texto de cada uno de los hijos
    y se ordenan de la siguiente manera dependiendo el caso:
        Cuando la cadena incluye tangente,seno, coseno, etc: Nodo actual, Cadena del hijo izquierdo, Cadena del hijo derecho
        En los demas casos: Cadena del hijo izquierdo, Nodo actual, Cadena del hijo derecho
'''
def obten_cadena(index,tree,cadena=''):
    nodo = tree[index]
    izq = ''
    der = ''
    funciones = ['atan','asin','acos','sqrt','sin','cos','tan','cot','csc','sec','exp','ln','log']
    if nodo.get('ind_h1',None):
        izq = obten_cadena(nodo['ind_h1'],tree,cadena)
    centro = simbolo_a_texto(tree[index]['val'])
    if nodo.get('ind_h2',None):
        der = obten_cadena(nodo['ind_h2'],tree,cadena)
    
    cadena = f'{izq} {centro} {der}' if not nodo['val'] in funciones else f'{centro} {izq} {der}'
    return cadena

In [77]:
ltx = "-\\frac{4 \\exp^{2}{\\left(y \\right)}}{3} - \\frac{4}{3}"
#ltx = "(900(x+22)/36)-(948/50144)/\\tan(x)"
#ltx = '\\sin{x}'
t,exp = procesa_cadena(ltx,symbs=['x','y'])
cadena = obten_cadena(0,t)
print(cadena)
voz = gTTS(text=cadena,lang='es',slow=False)
voz.save("math2speech.mp3")

   menos  4  por exponencial de  y   AL CUADRADO  entre  3  menos  4  entre  3 
